# Vehicle detection

Here we will play around with various algorithms, color spaces and other techniques for detecting vehicles in images.

In [1]:
# Imports
import cv2
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D